In [2]:
import numpy as np
import matplotlib.pyplot as plt
from molecule import CaOH, CaH, mu_N, gI
from scipy.constants import h, k
%matplotlib inline

import qls
import utils as ut

In [3]:
# Taken from Chou et al.

gj_list: list[float] = [-1.35, -1.35, -1.35, -1.34, -1.34, -1.34, -1.34, -1.34, -1.33, -1.33, -1.33, -1.32, -1.32, -1.31, -1.31]

cij_list: list[float] = [8.27, 8.26, 8.26, 8.26, 8.26, 8.25, 8.25, 8.24, 8.24, 8.23, 8.22, 8.21, 8.20, 8.19, 8.18]

b_field_gauss = 3.6
j_max = 14

# mo1 = CaH.create_molecule_data(b_field_gauss=b_field_gauss, j_max=j_max)
mo1 = CaH.create_molecule_data(b_field_gauss=b_field_gauss, j_max=j_max, gj_list = gj_list, cij_list = cij_list)

temperature = 300
states1 = qls.States(mo1, temperature)



# duration_us = 1000.0
# rabi_rate_mhz = 2*np.pi*0.005
# max_frequency_mhz = 0.05        # 50 kHz
# scan_points = 1000
# dephased = False
# coherence_time_us = 100
# is_minus = True

# pump_frequency_mhz = -0.002
# num_pumps = 200
# pump_dephased = True
# pump_rabi_rate_mhz = 2*np.pi*0.004
# pump_duration_us = 1000.0


# qls.apply_pumping(mo1, pump_frequency_mhz, num_pumps, pump_duration_us, pump_rabi_rate_mhz, pump_dephased, coherence_time_us, is_minus)




In [61]:
# I suppose that the molecule i give in input is not optically pumped. I do it inside the class, if it's possible

class BayesianStateEstimation:

    def __init__(self, model = None, temperature = 300, b_field_gauss = 3.6, j_max = 15):
        if model is None:
            model = CaOH(b_field_gauss==b_field_gauss, j_max=j_max)
        
        self.model = model

        # This always initializes thermally the molecule: any pumping applied before calling the Bayesian class will be destroyed
        states1 = qls.States(molecule = self.model, temperature = temperature)

        self.temperature = temperature
        self.init_prior()


    # prior is the column in the dataframe in the molecule
    def init_prior(self):
        self.prior = self.model.state_df["state_dist"]


    # optical pumping: updates the prior
    def optical_pumping(self, pump_frequency_mhz, num_pumps, pump_duration_us, rabi_rate_mhz, pump_dephased, coherence_time_us, is_minus):
        # this function changes the state_dist column of the molecule dataframe
        qls.apply_pumping(self.model, pump_frequency_mhz, num_pumps, pump_duration_us, rabi_rate_mhz, pump_dephased, coherence_time_us, is_minus)
        #for this reason, I update the prior
        self.prior = self.model.state_df["state_dist"]
        

    def measurement_setting(self, rabi_rate_mhz, dephased, coherence_time_us, is_minus):
        # the laser field fixes the rabi_rate: it's the same for the measurements that drive transitions and optical pumping.
        df_trans = self.model.transition_df

        measurements = [[df_trans.loc[df_trans["j"]==j].iloc[0]["energy_diff"] * 1e-3, 
                         np.pi/(rabi_rate_mhz*df_trans.loc[df_trans["j"]==j].iloc[0]["coupling"]), 
                         dephased, 
                         coherence_time_us,
                         is_minus
                         ] for j in range(1, j_max+1)]
        
        self.Probs_exc_list = []

        for frequency, duration, deph, coh_time, is_min in measurements:
            state_exc_probs = qls.get_excitation_probabilities(mo1, frequency, duration, rabi_rate_mhz, deph, coh_time, is_min)
            self.Probs_exc_list.append(state_exc_probs)


        

In [74]:
b_field_gauss = 3.27
j_max = 50

mo1 = CaOH.create_molecule_data(b_field_gauss=b_field_gauss, j_max=j_max)

temperature = 300
states1 = qls.States(mo1, temperature)

B = BayesianStateEstimation(model = mo1, temperature=temperature, b_field_gauss=b_field_gauss, j_max = j_max)

In [75]:
mo1.state_df[0:15]

,j,m,xi,spin_up,spin_down,zeeman_energy_khz,rotation_energy_ghz,state_dist
0,0,-0.5,False,0.000000,1.000000,6.961418,0.00,0.000885
1,0,0.5,True,1.000000,0.000000,-6.961418,0.00,0.000885
2,1,-1.5,False,0.000000,1.000000,6.126685,21.92,0.000882
3,1,-0.5,False,0.078672,-0.996901,7.044563,21.92,0.000882
4,1,0.5,False,0.070854,-0.997487,7.870990,21.92,0.000882
5,1,-0.5,True,0.996901,0.078672,-6.389296,21.92,0.000882
6,1,0.5,True,0.997487,0.070854,-7.036257,21.92,0.000882
7,1,1.5,True,1.000000,0.000000,-7.616685,21.92,0.000882
8,2,-2.5,False,0.000000,1.000000,5.291951,65.76,0.000876
9,2,-1.5,False,0.123594,-0.992333,6.312263,65.76,0.000876


In [76]:
B.model.state_df[0:15]

,j,m,xi,spin_up,spin_down,zeeman_energy_khz,rotation_energy_ghz,state_dist
0,0,-0.5,False,0.000000,1.000000,6.961418,0.00,0.000885
1,0,0.5,True,1.000000,0.000000,-6.961418,0.00,0.000885
2,1,-1.5,False,0.000000,1.000000,6.126685,21.92,0.000882
3,1,-0.5,False,0.078672,-0.996901,7.044563,21.92,0.000882
4,1,0.5,False,0.070854,-0.997487,7.870990,21.92,0.000882
5,1,-0.5,True,0.996901,0.078672,-6.389296,21.92,0.000882
6,1,0.5,True,0.997487,0.070854,-7.036257,21.92,0.000882
7,1,1.5,True,1.000000,0.000000,-7.616685,21.92,0.000882
8,2,-2.5,False,0.000000,1.000000,5.291951,65.76,0.000876
9,2,-1.5,False,0.123594,-0.992333,6.312263,65.76,0.000876


In [77]:
B.prior[0:15]

0     0.000885
1     0.000885
2     0.000882
3     0.000882
4     0.000882
5     0.000882
6     0.000882
7     0.000882
8     0.000876
9     0.000876
10    0.000876
11    0.000876
12    0.000876
13    0.000876
14    0.000876
Name: state_dist, dtype: float64

In [78]:
mo1.state_df["state_dist"][0:15]

0     0.000885
1     0.000885
2     0.000882
3     0.000882
4     0.000882
5     0.000882
6     0.000882
7     0.000882
8     0.000876
9     0.000876
10    0.000876
11    0.000876
12    0.000876
13    0.000876
14    0.000876
Name: state_dist, dtype: float64

In [ ]:
rabi_rate_mhz = 2*np.pi*0.005

# pumping
pump_frequency_mhz_1 = 0.00015
num_pumps = 1000
pump_duration_us = 1000.0
dephased = True
coherence_time_us = 100
is_minus = True


# this changes both the molecule mo1 dataframe (concerning the state_dist) and the B.model molecule (so also the prior)
B.optical_pumping(pump_frequency_mhz_1, num_pumps, pump_duration_us, rabi_rate_mhz, dephased, coherence_time_us, is_minus)

In [26]:
df_trans = B.model.transition_df

In [ ]:
[[df_trans.loc[df_trans["j"]==j].iloc[0]["energy_diff"] * 1e-3, np.pi/(rabi_rate_mhz*df_trans.loc[df_trans["j"]==j].iloc[0]["coupling"]) ] for j in range(1, j_max+1)]

[np.float64(-0.009850905598626464), np.float64(-0.013416052063320789), np.float64(-0.018779395792107963), np.float64(-0.02544750345747397), np.float64(-0.03274417507747613), np.float64(-0.04043015071799751), np.float64(-0.048229602655919274), np.float64(-0.05619997190711613), np.float64(-0.06414793513086044), np.float64(-0.0721212364266299), np.float64(-0.08010421505052599), np.float64(-0.08809191269585318), np.float64(-0.09607500058146846), np.float64(-0.10405252024050393)]


In [47]:
rabi_rate_mhz = 2*np.pi*0.005


measurs = [[df_trans.loc[df_trans["j"]==j].iloc[0]["energy_diff"] * 1e-3, np.pi/(rabi_rate_mhz*df_trans.loc[df_trans["j"]==j].iloc[0]["coupling"]) ] for j in range(1, j_max+1)]

In [54]:
print(measurs)

[[np.float64(-0.009850905598626464), np.float64(528.9557232377543)], [np.float64(-0.013416052063320789), np.float64(597.383776360106)], [np.float64(-0.018779395792107963), np.float64(766.5620904109643)], [np.float64(-0.02544750345747397), np.float64(996.1642510597642)], [np.float64(-0.03274417507747613), np.float64(1254.4941773044397)], [np.float64(-0.04043015071799751), np.float64(1526.4876832686357)], [np.float64(-0.048229602655919274), np.float64(1803.6030724352574)], [np.float64(-0.05619997190711613), np.float64(2085.259239514456)], [np.float64(-0.06414793513086044), np.float64(2366.588576532419)], [np.float64(-0.0721212364266299), np.float64(2648.6350679850475)], [np.float64(-0.08010421505052599), np.float64(2932.0242897460466)], [np.float64(-0.08809191269585318), np.float64(3214.6363877379)], [np.float64(-0.09607500058146846), np.float64(3498.310669171557)], [np.float64(-0.10405252024050393), np.float64(3781.0436578805607)]]


In [55]:
# Taken from Chou et al.

gj_list: list[float] = [-1.35, -1.35, -1.35, -1.34, -1.34, -1.34, -1.34, -1.34, -1.33, -1.33, -1.33, -1.32, -1.32, -1.31, -1.31]

cij_list: list[float] = [8.27, 8.26, 8.26, 8.26, 8.26, 8.25, 8.25, 8.24, 8.24, 8.23, 8.22, 8.21, 8.20, 8.19, 8.18]

b_field_gauss = 3.6
j_max = 14

# mo1 = CaH.create_molecule_data(b_field_gauss=b_field_gauss, j_max=j_max)
mo1 = CaH.create_molecule_data(b_field_gauss=b_field_gauss, j_max=j_max, gj_list = gj_list, cij_list = cij_list)

temperature = 300
states1 = qls.States(mo1, temperature)

duration_us = 766.5620904109643
rabi_rate_mhz = 2*np.pi*0.005
frequency = -0.018779395792107963      # 50 kHz
scan_points = 1000
dephased = False
coherence_time_us = 100
is_minus = True

state_exc_probs = qls.get_excitation_probabilities(mo1,frequency,duration_us,rabi_rate_mhz,dephased,coherence_time_us,is_minus)



In [44]:
sum(mo1.state_df["state_dist"])

1.0000000000000038

In [60]:
print(state_exc_probs)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.44318180e-03
 4.62389087e-03 1.12493456e-04 1.16414069e-03 2.17282308e-03
 0.00000000e+00 5.58711995e-03 1.19619074e-03 5.63263204e-04
 2.59288281e-03 2.68657350e-04 1.76907493e-03 1.49492476e-04
 5.99261159e-04 9.53191549e-04 0.00000000e+00 1.00000000e+00
 1.46961882e-04 1.13073064e-03 4.96318862e-05 6.60230164e-05
 1.61814185e-03 1.15484366e-03 4.09506385e-04 9.80107509e-04
 5.28525961e-05 6.53008546e-05 2.35427597e-07 3.03038807e-04
 0.00000000e+00 8.37373852e-04 4.97511931e-03 1.19727285e-03
 3.36417952e-05 5.04689859e-05 7.57513444e-04 3.01618645e-03
 4.21059112e-03 1.31081720e-03 1.32207875e-03 1.54569774e-03
 2.02207688e-04 1.02739585e-05 2.58779101e-05 3.84842647e-04
 1.28416038e-03 1.93106526e-03 0.00000000e+00 5.10072973e-04
 1.17468006e-03 4.05236356e-05 5.12735213e-04 2.68717996e-04
 2.04398964e-05 8.57521163e-04 2.55952027e-03 3.89205830e-03
 3.44126272e-03 1.42065874e-03 2.05346393e-03 4.81395810e-04
 4.61954578e-07 5.674235